# Movie Recommender Engine - 4
Data - Amazaon Movies and TV

## PySpark
PySpark powered Collaborative Filtering Approach to Recommendations

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

In [8]:
pwd

'/Users/siddharthmandgi/Desktop/2020-Summer-Honors-Home/Week 6'

In [56]:
df = pd.read_csv("/Users/siddharthmandgi/Desktop/2020-Summer-Honors-Home/Datasets/Amazon_Movies.csv")
df = df.rename(columns = {"Unnamed: 0": "User_index"})
df = df.rename(columns = {"asin" : "Movie_ID", "overall" : "ratings" })
df['ratings'] = df['ratings'].astype(int)
ratings = df['ratings'].unique().tolist()
df

,User_index,reviewerID,Movie_ID,reviewerName,helpful,reviewText,ratings,summary,unixReviewTime,reviewTime
0,0,ADZPIG9QOCDG5,0005019281,"Alice L. Larson ""alice-loves-books""","[0, 0]",This is a charming version of the classic Dick...,4,good version of a classic,1203984000,"02 26, 2008"
1,1,A35947ZP82G7JH,0005019281,Amarah Strack,"[0, 0]",It was good but not as emotionally moving as t...,3,Good but not as moving,1388361600,"12 30, 2013"
2,2,A3UORV8A9D5L2E,0005019281,Amazon Customer,"[0, 0]","Don't get me wrong, Winkler is a wonderful cha...",3,Winkler's Performance was ok at best!,1388361600,"12 30, 2013"
3,3,A1VKW06X1O2X7V,0005019281,"Amazon Customer ""Softmill""","[0, 0]",Henry Winkler is very good in this twist on th...,5,It's an enjoyable twist on the classic story,1202860800,"02 13, 2008"
4,4,A3R27T4HADWFFJ,0005019281,BABE,"[0, 0]",This is one of the best Scrooge movies out. H...,4,Best Scrooge yet,1387670400,"12 22, 2013"
...,...,...,...,...,...,...,...,...,...,...
1697528,1697528,AV657BUYHHXZ2,B00LT1JHLW,"Mike Rules ""Mike""","[1, 14]",wow $269.99 for the entire series on Blu Ray??...,1,Way to Expensive!! WB = GREED,1406073600,"07 23, 2014"
1697529,1697529,A17W587EH23J0Q,B00LT1JHLW,"Ron2900 ""Ron""","[32, 48]","Finally, the holy grail of tv-on-dvd boxsets i...",5,"HOLY BAT-BOXSET, BATMAN... I never thought thi...",1405641600,"07 18, 2014"
1697530,1697530,A3DE438TF1A958,B00LT1JHLW,thomas henry,"[3, 10]",Could this be a true or I'm i dreaming batman ...,5,prayers have been answered because batman 60s ...,1405728000,"07 19, 2014"
1697531,1697531,A2RWCXDMANY0LW,B00LT1JHLW,wheev,"[0, 4]",I've been a fan of the series since I was a yo...,5,can't Wait!,1405987200,"07 22, 2014"


In [57]:
def partition(x):
    if x<3:
        return 'Negative'
    elif x>3:
        return 'Positive'
    return 'Neutral'

actualScore = df['ratings']
positiveNegative = actualScore.map(partition)
df['sentiment'] = positiveNegative

In [58]:
import numpy as np
len(np.unique(df['Movie_ID']))

50052

In [74]:
df1 = df[['reviewerID','Movie_ID','ratings']]
df1 = df1.dropna() 

In [76]:
df1

,reviewerID,Movie_ID,ratings
0,ADZPIG9QOCDG5,0005019281,4
1,A35947ZP82G7JH,0005019281,3
2,A3UORV8A9D5L2E,0005019281,3
3,A1VKW06X1O2X7V,0005019281,5
4,A3R27T4HADWFFJ,0005019281,4
...,...,...,...
1697528,AV657BUYHHXZ2,B00LT1JHLW,1
1697529,A17W587EH23J0Q,B00LT1JHLW,5
1697530,A3DE438TF1A958,B00LT1JHLW,5
1697531,A2RWCXDMANY0LW,B00LT1JHLW,5


In [77]:
df1.to_csv('Amazon_Pyspark.csv',index=False)

In [78]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [79]:
MAX_MEMORY = "10g"

spark = SparkSession \
    .builder \
    .appName("Foo") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [80]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(spark)

In [81]:
#So you dont have to preporcess the data again and again
df_spark = spark.read.csv('/Users/siddharthmandgi/Desktop/2020-Summer-Honors-Home/Week 6/Amazon_Pyspark.csv', header = True)

In [82]:
df_spark.count()

1697533

In [103]:
df_spark.head()

Row(reviewerID='ADZPIG9QOCDG5', Movie_ID='0005019281', ratings=4)

In [84]:
df_spark.printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- Movie_ID: string (nullable = true)
 |-- ratings: string (nullable = true)



In [102]:
from pyspark.sql.types import IntegerType
df_spark = df_spark.withColumn("ratings", df_spark["ratings"].cast(IntegerType()))
df_spark.printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- Movie_ID: string (nullable = true)
 |-- ratings: integer (nullable = true)



In [104]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [105]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df_spark.columns)-set(['ratings'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(df_spark).transform(df_spark)
transformed.show()

+--------------+----------+-------+--------------+----------------+
|    reviewerID|  Movie_ID|ratings|Movie_ID_index|reviewerID_index|
+--------------+----------+-------+--------------+----------------+
| ADZPIG9QOCDG5|0005019281|      4|        3160.0|         88988.0|
|A35947ZP82G7JH|0005019281|      3|        3160.0|        110384.0|
|A3UORV8A9D5L2E|0005019281|      3|        3160.0|        116243.0|
|A1VKW06X1O2X7V|0005019281|      5|        3160.0|         99627.0|
|A3R27T4HADWFFJ|0005019281|      4|        3160.0|         13935.0|
|A2L0G56BNOTX6S|0005019281|      5|        3160.0|         80495.0|
| A5NYUBEKXFLX5|0005019281|      5|        3160.0|         26406.0|
|A2DJ8B8GE4V2VD|0005019281|      5|        3160.0|         79322.0|
| AWF2S3UNW9UA0|0005019281|      5|        3160.0|          1733.0|
|A3O4UUT83DG3OU|0005019281|      5|        3160.0|         86318.0|
|A3TK5Q142X5Q8A|0005019281|      5|        3160.0|        115993.0|
|A33EWPXESP9GQH|0005019281|      5|        3160.

In [106]:
(training,test)=transformed.randomSplit([0.8, 0.2])

In [107]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="reviewerID_index",itemCol="Movie_ID_index",ratingCol="ratings",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

In [109]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="ratings",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()

RMSE=1.1630469961379855
+--------------+----------+-------+--------------+----------------+----------+
|    reviewerID|  Movie_ID|ratings|Movie_ID_index|reviewerID_index|prediction|
+--------------+----------+-------+--------------+----------------+----------+
| ATS1HA5M3EPZ4|0783239408|      3|         148.0|           897.0| 3.8488202|
| AKMXCZXE0KD5H|0783239408|      5|         148.0|          3226.0| 4.4825144|
|A1W584W9UW3XEU|0783239408|      5|         148.0|         14423.0| 3.5754397|
| AE0E6UII0VSZA|0783239408|      5|         148.0|          2721.0|  4.509235|
|A2WN6UJP3KUDRH|0783239408|      5|         148.0|         28893.0| 4.4495144|
| AF1VHA7BVWLO1|0783239408|      5|         148.0|         12210.0| 4.5768123|
| AQ4NJ2T9PVI3L|0783239408|      5|         148.0|          2914.0|  3.437717|
|A1Q239XJPSI44W|0783239408|      1|         148.0|         22236.0| 2.5796769|
| AV0OBG2SS1I1R|0783239408|      5|         148.0|          4119.0| 4.6192846|
|A2NFTFQYW8GXIQ|0783239408| 

In [110]:
user_recs=model.recommendForAllUsers(20).show(10)

+----------------+--------------------+
|reviewerID_index|     recommendations|
+----------------+--------------------+
|             148|[[22102, 7.525032...|
|             463|[[42505, 6.882899...|
|             471|[[49301, 6.460155...|
|             496|[[49301, 7.932959...|
|             833|[[48592, 7.792800...|
|            1088|[[49301, 7.74932]...|
|            1238|[[47267, 6.549411...|
|            1342|[[49301, 8.127285...|
|            1580|[[49301, 7.297943...|
|            1591|[[47139, 7.110337...|
+----------------+--------------------+
only showing top 10 rows

